In [30]:
from paddleocr import PaddleOCR, draw_ocr # main OCR dependencies
from matplotlib import pyplot as plt # plot images
import cv2 #opencv
import os # folder directory navigation
import ollama

In [31]:
# Setup model
ocr_model = PaddleOCR(lang='en', use_gpu=False)


[2025/02/08 21:28:23] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\harsh/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\harsh/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=

In [44]:
img_path = os.path.join('.', 'C:\PF\Projects\LOC_7.0\images\Pine_bill.jpeg')

In [45]:
# Run the ocr method on the ocr model
result = ocr_model.ocr(img_path)

[2025/02/08 21:35:01] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2025/02/08 21:35:01] ppocr DEBUG: dt_boxes num : 36, elapsed : 0.3659780025482178
[2025/02/08 21:35:03] ppocr DEBUG: rec_res num  : 36, elapsed : 1.4262208938598633


In [46]:
result

[[[[[436.0, 14.0], [681.0, 8.0], [682.0, 54.0], [437.0, 60.0]],
   ('.ients', 0.7985985279083252)],
  [[[208.0, 164.0], [637.0, 179.0], [636.0, 211.0], [207.0, 196.0]],
   ('UPPER CRUST FOUDS UPPER CRUST FOUDS', 0.9368654489517212)],
  [[[379.0, 207.0], [459.0, 213.0], [456.0, 247.0], [377.0, 242.0]],
   ('MUMBAI', 0.9667284488677979)],
  [[[121.0, 238.0], [325.0, 244.0], [324.0, 276.0], [120.0, 270.0]],
   ('DATE:2025-02-07', 0.9436524510383606)],
  [[[536.0, 251.0], [709.0, 248.0], [709.0, 280.0], [536.0, 283.0]],
   ('TIME 16:41:49', 0.9174444675445557)],
  [[[122.0, 278.0], [370.0, 284.0], [369.0, 312.0], [121.0, 306.0]],
   ('MID 02PL00000512911', 0.9574407339096069)],
  [[[545.0, 287.0], [706.0, 287.0], [706.0, 315.0], [545.0, 315.0]],
   ('TID 20168971', 0.9576871991157532)],
  [[[125.0, 316.0], [334.0, 320.0], [334.0, 348.0], [124.0, 344.0]],
   ('BATCH NUM000055', 0.8671987056732178)],
  [[[511.0, 323.0], [704.0, 323.0], [704.0, 349.0], [511.0, 349.0]],
   ('INV.NUM047631', 0.

In [47]:
def get_strings_simple(data):
    """Extract strings using simple list comprehension"""
    return [item[1] for item in data[0]]


In [23]:
def query_qwen2(text):
    """Sends text to the locally running Qwen2.5-3B model and forces valid JSON output."""
    prompt = f"""
    Extract structured details from the following receipt text. Respond only in JSON format without any extra text.
    **Input Receipt Text:**
    \"\"\"{text}\"\"\"
    **JSON Format (Example Output):**
    {{
        "vendor": "Walmart",
        "amount": "45.67",
        "date": "2024-02-08",
        "category": "Grocery"
    }}
    Now, extract details from the given receipt and return only JSON:
    """
    response = ollama.chat(model='qwen2.5:3b', messages=[{"role": "user", "content": prompt}])
    json_text = response['message']['content'].strip()
    # Ensure only valid JSON is returned
    try:
        structured_data = json.loads(json_text)  # Parse JSON response
        return structured_data
    except json.JSONDecodeError:
        print("Error: Model response is not valid JSON.\nResponse:\n", json_text)
        return None

In [24]:
def process_receipt(ocr_text_list):
    """Takes OCR-extracted text and processes it using Qwen2.5-3B to extract structured receipt details."""
    combined_text = " ".join([text[0] for text in ocr_text_list])  # Combine OCR-extracted text
    structured_data = query_qwen2(combined_text)  # Send to Qwen model
    return structured_data

In [26]:
# Read and process image
result = ocr_model.ocr(img_path)

[2025/02/08 18:40:04] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2025/02/08 18:40:05] ppocr DEBUG: dt_boxes num : 53, elapsed : 0.05650210380554199
[2025/02/08 18:40:06] ppocr DEBUG: rec_res num  : 53, elapsed : 1.6820707321166992


In [48]:
# Extract strings
strings = get_strings_simple(result)

print(strings)

[('.ients', 0.7985985279083252), ('UPPER CRUST FOUDS UPPER CRUST FOUDS', 0.9368654489517212), ('MUMBAI', 0.9667284488677979), ('DATE:2025-02-07', 0.9436524510383606), ('TIME 16:41:49', 0.9174444675445557), ('MID 02PL00000512911', 0.9574407339096069), ('TID 20168971', 0.9576871991157532), ('BATCH NUM000055', 0.8671987056732178), ('INV.NUM047631', 0.9153614044189453), ('Sale', 0.9975618720054626), ('2', 0.541294515132904), ('***********6969', 0.9812787771224976), ('Chip', 0.9989089369773865), ('FXP DATE XX/XX', 0.8773998022079468), ('CARD TYPEMASTERCARD', 0.9653713703155518), ('TXN ID', 0.870819091796875), ('6271715955', 0.9737598299980164), ('APPMastercard', 0.9935408234596252), ('IDA0000000041010', 0.9672988653182983), ('TVR:0000048000', 0.9667589068412781), ('SIE800', 0.8819334506988525), ('TC3AEC3FCA7552EEF0', 0.9541875123977661), ('APPR CODE R02594', 0.9168578386306763), ('RRN005690228685', 0.9634175896644592), ('BASE AMT.RS', 0.9287031292915344), ('1176.00', 0.9971885681152344), ('

In [29]:
# Process receipt
receipt_details = process_receipt(strings)
print("Extracted Receipt Details:", receipt_details)

Extracted Receipt Details: {'vendor': 'Walmart', 'amount': '49.90', 'date': '2024-02-08', 'category': 'Grocery'}
